# Simulations for <br> Group FeAR
---


- The following notebook can be used to run GWorld simulations and to evaluate the FeAR values in each instance for group responsibilities. 

### Select the Scenario for the simulation
- Scenarios describe the map of the gworld and location and behaviour of agents
- Scenarios for the gFeAR paper are stored in `Scenarios4FeARSims.json`
- Uncomment the relevant lines to select the scenario for the simulation.
- To generate new scenarios, use the notebook `ScenarioBuilder.ipynb`

In [ ]:
# Selecting the scenario

scenario_name = 'Elementary'
# scenario_name ='CrossyRoad1'
# scenario_name = 'ProveHelper2'

# Randomised simulations - ! Long run times !
# scenario_name = 'Intersection12by12-8Agents-50Cases-5Iterations-policy-aggressive'
# scenario_name = 'Intersection12by12-8Agents-50Cases-5Iterations-policy-nuanced'
# scenario_name = 'Intersection12by12-8Agents-50Cases-5Iterations-policy-random'


### Basic Settings
  - `ShowPlots4Instance` - Whether plots to be made for instance of the simulation (Might increase runtime).
  - `PlotGroupFeARs` - whether to plot all the group FeAR values (Lots of images!)

In [ ]:
ShowPlots4Instance = True
# ShowPlots4Instance = False

PlotGroupFeARs = True
# PlotGroupFeARs = False

SaveImagestoFolder = 'Plots'
# SaveImagestoFolder = ''

results_folder = 'gFeAR_Results'

### Imports and Setup

In [ ]:
%%capture
%matplotlib inline


import numpy as np; np.random.seed(0)
np.set_printoptions(precision=2)
import seaborn as sns; sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation,ArtistAnimation
import matplotlib.animation as animation
import copy
from scipy.stats import entropy as entropy
from tqdm import tqdm
rng = np.random.default_rng(seed=0)
import json
from pprint import pprint
import datetime
import os
import pandas as pd
import datetime
from itertools import permutations

import GWorld
import Agent
import Emergence
import PlotGWorld
import Responsibility
import GroupSort 
from GroupSort import GroupSortingAlgorithm, generate_tier_tikz, generate_tier_mermaid, get_agent_ranks_from_tiers,get_agent_ranks_from_fear

plotgw = PlotGWorld.PlotGWorld(); # Object for accessing plotters

import AnalysisOf_FeAR_Sims as FeARUI


### Select the Moves de Rigueur

Use this cell to set:
- `MdR4Agents_Default`
- `Specific_MdR4Agents`

For gFeAR paper MdRs were set as stay.

In [ ]:
# Setting the Moves de Rigeuer

# Default MdR for all the agents
# ----------------------------
MdR4Agents_Default = 0 #Stay
# MdR4Agents_Default = 4 #Right1
# MdR4Agents_Default = 8 #Right2
# MdR4Agents_Default = 12 #Right3
# MdR4Agents_Default = 16 #Right4
#Up Down Left Right

# Define specific MdR for specific Agents
# Syntax: Specific_MdR4Agents = [[AgentID,MDR], [AgentID,MdR], ...]
# ------------------------------------------------------------------
Specific_MdR4Agents = [] #None
# Specific_MdR4Agents = [[3,0], [4, 3], [5, 2]]
# Specific_MdR4Agents = [[2,6]]


#### All inputs finalized !

---

---


### Running GWorld

- All the relevant settings have been set in the preceding cells.
- **All the following cells can be run altogether.** 

#### Get scenarios from JSON file

In [ ]:
# Get Scenarios from JSON file
Scenario = GWorld.LoadJsonScenario(json_filename='Scenarios4FeARSims.json',scenario_name=scenario_name)

N_Agents = Scenario['N_Agents']
N_Cases = Scenario['N_Cases']
N_iterations = Scenario['N_iterations']

# Just a check - Minimum one iteration
if N_iterations <= 0:
    N_iterations = 1

ActionNames,ActionMoves = Agent.DefineActions()

print('N_Agents : ',N_Agents)
print('N_Cases : ',N_Cases)
print('N_iterations : ',N_iterations)

#### Preview GWorld

In [ ]:
# GWorld Preview
Region = np.array(Scenario['Map']['Region'])
Walls = Scenario['Map']['Walls']
OneWays = Scenario['Map']['OneWays']

World = GWorld.GWorld(Region, Walls= Walls, OneWays = OneWays) # Initialising GWorld from Scenario

N_Agents = Scenario['N_Agents']
AgentLocations = Scenario['AgentLocations'].copy()
    
    
AgentLocations = []
for location in Scenario['AgentLocations']:
    AgentLocations.append(tuple(location))

# Adding N Agents at sorted random positions
if len(AgentLocations) < N_Agents:
    [locX,locY] = np.where(Region==1)

    LocIdxs = rng.choice(locX.shape[0], size=(N_Agents-len(AgentLocations)), replace=False, shuffle=False)
    LocIdxs.sort()

    for Idx in LocIdxs:
        AgentLocations.append((locX[Idx],locY[Idx]))

# Adding Agents
PreviousAgentAdded = True
for location in AgentLocations:
    # Adding new Agents if Previous Agent was Added to the World
    if PreviousAgentAdded: 
        Ag_i = Agent.Agent()
    PreviousAgentAdded = World.AddAgent(Ag_i,location, printStatus=False)

PreviousAgentAdded = True
while len(World.AgentList) < N_Agents:
    # Adding new Agents if Previous Agent was Added to the World
    if PreviousAgentAdded: 
        Ag_i = Agent.Agent()
    Loc_i = (np.random.randint(Region.shape[0]),np.random.randint(Region.shape[1]))
    PreviousAgentAdded = World.AddAgent(Ag_i,Loc_i, printStatus=False)

# -------------------------------------------------------------------------------
# Selecting actions for agents
# -------------------------------------------------------------------------------

defaultAction = Scenario['defaultAction']
SpecificAction4Agents = Scenario['SpecificAction4Agents']

# Setting Policy for all Agents

# The default Step and Direction Weights
StepWeights=Scenario['StepWeights']
DirectionWeights=Scenario['DirectionWeights']

ListOfStepWeights = []
ListOfDirectionWeights = []

for ii in range(len(World.AgentList)):
    ListOfStepWeights.append(StepWeights)
    ListOfDirectionWeights.append(DirectionWeights)


# Updating the list of stepweights based on specific weights for agents    
for agentIDs,stepweights4agents in Scenario['SpecificStepWeights4Agents']:
    for agentID in agentIDs:
        ListOfStepWeights[agentID] = stepweights4agents

# Updating the list of directionweights based on specific weights for agents            
for agentIDs,directionweights4agents in Scenario['SpecificDirectionWeights4Agents']:
    for agentID in agentIDs:
        ListOfDirectionWeights[agentID] = directionweights4agents

# Updating Agent Policies in World   
    for ii,agent in enumerate(World.AgentList):
        policy = Agent.GeneratePolicy(StepWeights=ListOfStepWeights[ii],DirectionWeights=ListOfDirectionWeights[ii])
        agent.UpdateActionPolicy(policy)

if 'Policies' in (Scenario.keys()):
    # Dictionary of Policies
    if Scenario['Policies']:
        policies = Scenario['Policies']
        policy_map = np.zeros(np.shape(Region), dtype=int)
        
        policy_keys = Scenario['Policies'].keys()
        # print(f'{policy_keys =}')
        for key in policy_keys:
            slicex = Scenario['Policies'][key]['slicex']
            slicey = Scenario['Policies'][key]['slicey']
            policy_map[slicex, slicey] = key
    
        print(f'Region =\n {Region}')
        print(f'policyMap =\n {policy_map}')
    
        # Setting Policy for all Agents
    
        # Updating Agent Policies in World
        for ii, agent in enumerate(World.AgentList):
            agent_location = World.AgentLocations[ii]
            agent_policy = str(policy_map[agent_location[0], agent_location[1]]).zfill(2)
            if 'policyWeights' in policies[agent_policy].keys():
                policy_weights = policies[agent_policy]['policyWeights']
                agent_stepWeights = None
                agent_directionWeights = None
            else:
                policy_weights = None
                agent_stepWeights = policies[agent_policy]['stepWeights']
                agent_directionWeights = policies[agent_policy]['directionWeights']
    
            policy = Agent.GeneratePolicy(StepWeights=agent_stepWeights, DirectionWeights=agent_directionWeights, policy_weights=policy_weights)        
            agent.UpdateActionPolicy(policy)


Action4Agents = World.SelectActionsForAll(defaultAction = defaultAction,InputActionID4Agents = SpecificAction4Agents)

# Plotting the State of the World and Chosen Actions for the next iteration
plotgw.ViewGWorld(World, ViewNextStep = True ,ViewActionTrail=False);

#### Running GWorld Instances for the Scenario

- Simulation run for `N_Cases`.
- Each case consists of a sequence of `N_iterations`.
- $\therefore$ Total number of instances per scenario = `N_Cases` $\times$ `N_iterations`

**WARNING!** - The following is the cell that requires the longest to run.



In [ ]:
from matplotlib.colors import LinearSegmentedColormap

rank_summaries = []

# Runnning Simulation Cases !
maxCaseDigits = np.ceil(np.log10(N_Cases*N_iterations)).astype(int)

for jj in range(N_Cases):


    # Initialising World Map

    Region = np.array(Scenario['Map']['Region'])
    Walls = Scenario['Map']['Walls']
    OneWays = Scenario['Map']['OneWays']


    World = GWorld.GWorld(Region, Walls= Walls, OneWays = OneWays) # Initialising GWorld from Matrix A

    AgentLocations = Scenario['AgentLocations'].copy()
    AgentLocations = []
    for location in Scenario['AgentLocations']:
        AgentLocations.append(tuple(location))
    
    # Adding N Agents at sorted random positions
    if len(AgentLocations) < N_Agents:
        [locX,locY] = np.where(Region==1)

        LocIdxs = rng.choice(locX.shape[0], size=(N_Agents-len(AgentLocations)), replace=False, shuffle=False)
        LocIdxs.sort()

        for Idx in LocIdxs:
            AgentLocations.append((locX[Idx],locY[Idx]))

    # Adding Agents
    PreviousAgentAdded = True
    for location in AgentLocations:
        # Adding new Agents if Previous Agent was Added to the World
        if PreviousAgentAdded: 
            Ag_i = Agent.Agent()
        PreviousAgentAdded = World.AddAgent(Ag_i,location, printStatus=False)

    PreviousAgentAdded = True
    while len(World.AgentList) < N_Agents:
        # Adding new Agents if Previous Agent was Added to the World
        if PreviousAgentAdded: 
            Ag_i = Agent.Agent()
        Loc_i = (np.random.randint(Region.shape[0]),np.random.randint(Region.shape[1]))
        PreviousAgentAdded = World.AddAgent(Ag_i,Loc_i, printStatus=False)


    # Action Selection for Agents
    defaultAction = Scenario['defaultAction']
    SpecificAction4Agents = Scenario['SpecificAction4Agents']
        
    #------------------------------------------------------------------------------------------------------------------

    # Setting Policy for all Agents

    # The default Step and Direction Weights
    StepWeights=Scenario['StepWeights']
    DirectionWeights=Scenario['DirectionWeights']

    ListOfStepWeights = []
    ListOfDirectionWeights = []

    for ii in range(len(World.AgentList)):
        ListOfStepWeights.append(StepWeights)
        ListOfDirectionWeights.append(DirectionWeights)


    # Updating the list of stepweights based on specific weights for agents    
    for agentIDs,stepweights4agents in Scenario['SpecificStepWeights4Agents']:
        for agentID in agentIDs:
            ListOfStepWeights[agentID] = stepweights4agents

    # Updating the list of directionweights based on specific weights for agents            
    for agentIDs,directionweights4agents in Scenario['SpecificDirectionWeights4Agents']:
        for agentID in agentIDs:
            ListOfDirectionWeights[agentID] = directionweights4agents

    # Updating Agent Policies in World   
    for ii,agent in enumerate(World.AgentList):
        policy = Agent.GeneratePolicy(StepWeights=ListOfStepWeights[ii],DirectionWeights=ListOfDirectionWeights[ii])
        agent.UpdateActionPolicy(policy)

    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------


    # Move de Rigueur
   
    # MdR4Agents = [[1,4]]
    MdR4Agents = []

    #Setting the MdR for each Agent
    for ii in range(len(World.AgentList)):
        MdR4Agents.append([ii, MdR4Agents_Default])
        
    for agent,specific_mdr in Specific_MdR4Agents:
        MdR4Agents[agent] = [agent, specific_mdr]

    print('MdR4Agents : ',MdR4Agents)
    mdr_string = FeARUI.get_mdr_string(MdR4Agents, return_names=True)
    print('MdRs: ', mdr_string)

    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------

    for kk in range(N_iterations): 
    
        InstanceID = 'Instance_' + str(jj*N_iterations+kk).zfill(maxCaseDigits)
        print('Instance : ',InstanceID)
        print('---------------------------------------------\n')

        #------------------------------------------------------------------------------------------------------------------

        # Responsibility

        if 'Policies' in (Scenario.keys()):
            # Dictionary of Policies
            if Scenario['Policies']:
                policies = Scenario['Policies']
                policy_map = np.zeros(np.shape(Region), dtype=int)
                
                policy_keys = Scenario['Policies'].keys()
                # print(f'{policy_keys =}')
                for key in policy_keys:
                    slicex = Scenario['Policies'][key]['slicex']
                    slicey = Scenario['Policies'][key]['slicey']
                    policy_map[slicex, slicey] = key
            
                print(f'Region =\n {Region}')
                print(f'policyMap =\n {policy_map}')
            
                # Setting Policy for all Agents
            
                # Updating Agent Policies in World
                for ii, agent in enumerate(World.AgentList):
                    agent_location = World.AgentLocations[ii]
                    agent_policy = str(policy_map[agent_location[0], agent_location[1]]).zfill(2)
                    if 'policyWeights' in policies[agent_policy].keys():
                        policy_weights = policies[agent_policy]['policyWeights']
                        agent_stepWeights = None
                        agent_directionWeights = None
                    else:
                        policy_weights = None
                        agent_stepWeights = policies[agent_policy]['stepWeights']
                        agent_directionWeights = policies[agent_policy]['directionWeights']
            
                    policy = Agent.GeneratePolicy(StepWeights=agent_stepWeights, DirectionWeights=agent_directionWeights, policy_weights=policy_weights)
                    agent.UpdateActionPolicy(policy)
            
        # Select Actions for Agents based on defaultAction and SpecificAction4Agents
        Action4Agents = World.SelectActionsForAll(defaultAction = defaultAction,InputActionID4Agents = SpecificAction4Agents)

        # Calculate Responsibility Metric for the chosen Actions
        
        tr = []
        tmdr = []
        tva = []
        individual_fear = []

        if len(World.AgentList) >3:
            actors_on_x = False
            annot_font_size = 16
        else:
            actors_on_x = True     
            annot_font_size = 18

        # -------------------------------------------------------------------------------------------------------------
        # Calculate Shapley Values
        # -------------------------------------------------------------------------------------------------------------
        shapley_values = Responsibility.ShapleyValue(world=World, action_id_4agents=Action4Agents, mdr4agents=MdR4Agents)
        
        # Print detailed view with bar charts
        Responsibility.print_shapley_values(shapley_values)
        # Print matrix view
        Responsibility.print_shapley_values_table(shapley_values)
        #------------------------------------------------------------------------------------------------------------------
        #------------------------------------------------------------------------------------------------------------------

        for i in range(1, len(World.AgentList)):
            resp, num_v_mdr, num_v_a, = Responsibility.GroupResponsibility(World, Action4Agents, i, MdR4Agents)
            if i == 1:
                individual_fear = resp
            tr.append(resp)
            tmdr.append(num_v_mdr)
            tva.append(num_v_a)

            if PlotGroupFeARs:
                PlotGWorld.plotGroup(i=i, n=len(World.AgentList), resp=resp, for_print=True, annot_font_size=annot_font_size,
                                     actors_on_x=actors_on_x,
                                     saveFolder = SaveImagestoFolder, scenario_name=scenario_name, overwrite_image=True)        
   
        group_algorithm = GroupSortingAlgorithm(World, Action4Agents, MdR4Agents)

        print(f'{GWorld.get_feasibile_actions_for_affected_tuple.cache_info()=}')
        
        # for i in [0]:
        for i in range(0, len(World.AgentList)):
        
            individual_fear_on_affected = individual_fear[i]
            sorted_indices = np.argsort(individual_fear_on_affected)[::-1]
            sorted_fears = individual_fear_on_affected[sorted_indices]
            non_nan_mask = ~np.isnan(sorted_fears)
            filtered_sorted_fears = sorted_fears[non_nan_mask]
            filtered_sorted_indices = sorted_indices[non_nan_mask]

            tier_group, tier_fear = group_algorithm.find_tier_structure(filtered_sorted_fears, filtered_sorted_indices, i)
            GroupSort.print_group_tiers_for_affected(tier_group=tier_group,tier_fear=tier_fear, affected=i)

            rank_summary = GroupSort.find_and_compare_ranks(
                n_agents=len(World.AgentList), affected=i,
                tier_group=tier_group,
                individual_fear=individual_fear,
                shapley_values=shapley_values,
                scenario_name=scenario_name,
                return_rank_summary=True)
            
            rank_summary['scenario_name'] = scenario_name
            rank_summary['AgentLocations'] = np.array(World.AgentLocations)
            rank_summary['Action4Agents']= np.array(Action4Agents)
            rank_summary['MdR4Agents']= MdR4Agents

            rank_summaries.append(rank_summary)
            
            if tier_group:
                _ = generate_tier_tikz(tier_group, affected=i, scenario_name=scenario_name, save_tex=True)
                
        print(f'{GWorld.get_feasibile_actions_for_affected_tuple.cache_info()=}')
        
        FeAR,ValidMoves_MdR,ValidMoves_action1,ValidityOfMoves_Mdr,ValidityOfMoves_action1 = Responsibility.FeAR(World, Action4Agents,MdR4Agents)
        FeAL, ValidMoves_moveDeRigueur_FeAL, ValidMoves_action_FeAL, \
           ValidityOfMoves_Mdr_FeAL, ValidityOfMoves_action_FeAL = Responsibility.FeAL(World, Action4Agents,MdR4Agents)
        

        # Plot results
        if ShowPlots4Instance == True:

            # Plotting the State of the World and Chosen Actions for the next iteration
            if len(World.AgentList) >5:
                annot_font_size = 24
            else:
                annot_font_size = 24

            plotgw.ViewGWorld(World,ViewNextStep = True,ViewActionTrail=False, annot_font_size=annot_font_size,
                              saveFolder = SaveImagestoFolder, imageName=f'{scenario_name}_scenario', overwrite_image=True)

            for AgentID,ActionID in Action4Agents:
                print('Agent:',AgentID+1,'Action',ActionNames[ActionID],ActionID)

            # Showing the Responsibility Metric
            finer=False
            if len(World.AgentList) > 5:
                finer=True
                # _, fear_ax = plt.subplots(figsize=(5, 5), dpi=200)
                # _, shap_ax = plt.subplots(figsize=(5, 5), dpi=200)
                # annot_font_size = 12
                _, fear_ax = plt.subplots(figsize=(4, 4), dpi=200)
                _, shap_ax = plt.subplots(figsize=(4, 4), dpi=200)
                annot_font_size = 14
                finer=False
            else:
                _, fear_ax = plt.subplots(figsize=(2, 2), dpi=200)
                _, shap_ax = plt.subplots(figsize=(2, 2), dpi=200)
                annot_font_size = 18
                finer=False
            
            PlotGWorld.plotResponsibility(FeAR, FeAL=FeAL, fmt='.1f', ax=fear_ax, title='FeAR', annot_font_size=annot_font_size,
                                         imageName=f'{scenario_name}_fear', saveFolder = SaveImagestoFolder,
                                          for_print=False, save_file=True, finer=finer, overwrite_image=True);
            if PlotGroupFeARs:
                _, fear_g_ax = plt.subplots(figsize=(max(len(World.AgentList),2), max(len(World.AgentList)/2,1.7)))
                PlotGWorld.plotResponsibility(FeAR, FeAL=FeAL, fmt='.2f', ax=fear_g_ax, title='FeAR', annot_font_size=annot_font_size,
                                             imageName=f'{scenario_name}_fear_squished', saveFolder = SaveImagestoFolder,
                                              for_print=False, finer=False,
                                              square=False, save_file=True, overwrite_image=True);
            
            PlotGWorld.plotResponsibility(shapley_values.T, fmt='.1f', ax=shap_ax, title='Shapley Values', annot_font_size=annot_font_size,
                                         imageName=f'{scenario_name}_shap', saveFolder = SaveImagestoFolder,
                                          for_print=False, save_file=True, finer=finer, overwrite_image=True);

            
            fig,axs = plt.subplots(1,2)
    
            PlotGWorld.plotCounts(ValidMoves_MdR,title='Valid Moves of Affected if Actor MdR', ax=axs[0], cbar=False)
            PlotGWorld.plotCounts(ValidMoves_action1,title='Valid Moves of Affected if Actor Acts', ax=axs[1], cbar=False)
            
            plt.show()

        #------------------------------------------------------------------------------------------------------------------'
        #      Update GWorld
        #------------------------------------------------------------------------------------------------------------------        

        # Update World with Selected Steps
        agent_crashes, restricted_moves = World.UpdateGWorld(ActionID4Agents=Action4Agents)
        
        print('-' * 80)
        print('Updated GWorld !')
        print('-' * 80)
        GWorld.agent_crash_report(agent_crashes=agent_crashes, restricted_moves=restricted_moves)
        print('-' * 80)
        
        # Plot updated state of GWorld
        plotgw.ViewGWorld(World,ViewActionArrows=False ,ViewActionTrail=True)

    del World

    #------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------------------------------------------------------------------

    ranks_df = pd.DataFrame(rank_summaries)


#### Done

In [ ]:
print('Done!')

### Saving rank summaries as df

In [ ]:
display(ranks_df)

t = datetime.datetime.now()
time_string = t.strftime('%Y-%m-%d_%H-%M-%S')
df_filename= os.path.join(results_folder, f'{scenario_name}_{time_string}_ranks.pkl')

ranks_df.to_pickle(df_filename)

### Plot summary of Kendall's Taus comparing ranks

In [ ]:
tau_df_ = ranks_df[['tau_fear_tier','tau_fear_shap','tau_tier_shap']]
tau_df = pd.melt(tau_df_,
                 value_vars=['tau_fear_tier','tau_fear_shap','tau_tier_shap'],
                 var_name = 'comparison',
                 value_name='tau'
                )

# Clean up the letter column to remove 'tau_' prefix
tau_df['comparison'] = tau_df['comparison'].str.replace('tau_', '')
# display(tau_df)

ax= sns.boxplot(tau_df, y='tau', x='comparison', hue='comparison', gap=0.1, palette="vlag" )
PlotGWorld.make_axes_gray(ax)

